In [7]:
# Pandas is a package containing additional functions to use data frames in Python
import pandas as pd

# These two lines allow the notebook to access the Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# This is the path to the project folder within the Google Drive.
file_path = "/content/drive/My Drive/"

import warnings
warnings.simplefilter('ignore')

import random

Mounted at /content/drive


---
## Notebook 2

# Data Cleaning - Automated

Now we can run the data cleaning steps on all the tables.  We can automate this by reading all the species names from a file (in our Drive as species_names.tsv) into a Python list.

In [15]:
species_list = [line.strip() for line in open(file_path + "species_names.tsv")]

In [16]:
species_list

['Acromyrmex_echinatior',
 'Amblyomma_americanum',
 'Amblyomma_aureolatum',
 'Amblyomma_sculptum',
 'Apis_cerana',
 'Apis_mellifera',
 'Asobara_tabida',
 'Athalia_rosae',
 'Biorhiza_pallida',
 'Bombus_terrestris',
 'Camponotus_castaneus',
 'Camponotus_floridanus',
 'Camponotus_japonicus',
 'Camponotus_ligniperdus',
 'Cardiocondyla_obscurior',
 'Cephus_cinctus',
 'Ceratina_australensis',
 'Cotesia_vestalis',
 'Crematogaster_osakensis',
 'Dermacentor_andersoni',
 'Dermacentor_variabilis',
 'Dinoponera_quadriceps',
 'Exoneurella_tridentata',
 'Fopius_arisanus',
 'Halictus_scabiosae',
 'Harpegnathos_saltator',
 'Ixodes_persulcatus',
 'Ixodes_ricinus',
 'Ixodes_scapularis',
 'Lasius_niger',
 'Linepithema_humile',
 'Lysiphlebus_fabarum',
 'Megachile_rotundata',
 'Megalopta_genalis',
 'Megastigmus_spermotrophus',
 'Messor_barbarus',
 'Messor_capitatus',
 'Messor_hellenius',
 'Messor_structor',
 'Microplitis_demolitor',
 'Monomorium_pharaonis',
 'Nasonia_giraulti',
 'Nasonia_vitripennis',
 'Ne

Using a ```for``` loop we can go through each of these names one by one and run all the data cleaning steps.

This is exactly the same code as we used for a single table - I have just moved it all into one cell to make it easier to run the loop.

I have added two extra lines so that we can combine all the single row summary tables into one long table for all species.

I also added one extra step - where there are more than 10,000 records we take a random sample of 10,000 - otherwise the georeferencing step takes too long.


In [10]:
# create an empty DataFrame to store all the summary table rows
all_summary_tables = pd.DataFrame()

# run everything for every species in the list
for species_name in species_list:
      # Everything in this indented section will run once for each species name

      # Read the input table from GBIF into Python.
      mytab = pd.read_csv(file_path + "/species_tables/" + species_name + ".csv", sep="\t")

      # Make a "summary" table to record information about the data as we go along.
      # Create an empty dataframe with these columns and with one row for this species
      summary = pd.DataFrame(columns=['nrecords_unfiltered', 'species', 'genus', 'family', 'order', 'class', 'phylum', 'kingdom', 'wrong_taxon_rank_count',
                                      'no_country_code_count', 'no_latlong_count', 'total_removed_data_cleaning', 'nrecords_clean', 'nrecords_after_data_cleaning'],
                            index=[species_name])
      # Count the number of records for this species in the unfiltered input table
      # Record this information in the summary table and write it to the log file.

      nrecords = len(mytab)
      summary.loc[species_name, 'nrecords_unfiltered'] = nrecords

      # Count the number of unique values in the “kingdom”, “phylum”, “class”, “order”, “family”, “genus” and “species” columns - there should only be one in each.
      # Print a warning if there is more than one.
      # I have removed some of the print statements here just to reduce the amount of output

      # if there is only one kingdom for all the rows
      if mytab['kingdom'].nunique() == 1:
        # record the first value in the kingdom column (as they are all the same)
        kingdom = mytab['kingdom'].values[0]
      # if there is > 1
      else:
        # write this message to the log file
        print("Warning - the kingdom column for %s has multiple values" % species_name)
        # record the kingdom for this species as "NA"
        kingdom = "NA"
      summary.loc[species_name, "kingdom"] = kingdom

      # repeat for phylum, class, order, family, genus, species
      if mytab['phylum'].nunique() == 1:
        phylum = mytab['kingdom'].values[0]
      else:
        print("Warning - the phylum column for %s as multiple values" % species_name)
        phylum = "NA"
      summary.loc[species_name, "phylum"] = phylum

      if mytab['class'].nunique() == 1:
        clas = mytab['class'].values[0]
      else:
        print("Warning - the class column for %s has multiple values" % species_name)
        clas = "NA"
      summary.loc[species_name, "class"] = clas

      if mytab['order'].nunique() == 1:
        order = mytab['order'].values[0]
      else:
        print("Warning - the order column for %s has multiple values" % species_name)
        order = "NA"
      summary.loc[species_name, "order"] = order

      if mytab['family'].nunique() == 1:
        family = mytab['family'].values[0]
      else:
        print("Warning - the family column for %s has multiple values" % species_name)
        family = "NA"
      summary.loc[species_name, "family"] = family

      if mytab['genus'].nunique() == 1:
        genus = mytab['genus'].values[0]
      else:
        print ("Warning - the genus column for %s has multiple values" % species_name)
        genus = "NA"
      summary.loc[species_name, "genus"] = genus

      if mytab['species'].nunique() == 1:
        species = mytab['species'].values[0]
      else:
        print("Warning - the species column for %s has multiple values" % species_name)
        species = "NA"
      summary.loc[species_name, "species"] = species


      filtered_tab_subspecies_only = mytab[mytab["taxonRank"] == "SUBSPECIES"]
      filtered_tab_subspecies_only.to_csv(file_path + "/subspecies_tables/" + species_name + ".csv", sep="\t", index=None)

      # count how many rows in the unfiltered table have something other than species in this column
      count_wrong_taxon_rank = len(mytab[mytab['taxonRank'] != "SPECIES"])
      # record this count in the summary table
      summary.loc[species_name, "wrong_taxon_rank_count"] = count_wrong_taxon_rank

      # In the taxonRank column, some records are classified as “SPECIES” and some as SUBSPECIES”.
      # Subspecies is a more specific classification - some researchers will be able to recognise and record different subspecies and others will not. 
      # For now, we will focus on the “SPECIES” records because there are more of them.

      # Create and save a smaller table of individuals with “SUBSPECIES” in this column.

      # Remove these individuals from the main table - filter it to keep only records where taxonRank == “SPECIES”.

      # filter out all rows which don't have "SPECIES" in this column
      mytab = mytab[mytab["taxonRank"] == "SPECIES"]

      # Create and save a smaller table of samples with no value in the “countryCode” column
      # then remove these individuals from the main table.

      filtered_table_null = mytab[mytab['countryCode'].isnull()]
      filtered_table_null.to_csv(file_path + "/species_tables_null/" + species_name + ".csv", sep="\t", index=None)

      # count how many rows have no country code
      count_no_country_code = len(mytab[mytab['countryCode'].isnull()])
      summary.loc[species_name, "no_country_code_count"] = count_no_country_code

      mytab = mytab[mytab['countryCode'].notnull()]

      # Remove rows where latitude or longitude is NA
      count_no_latlong = len(mytab[(mytab['decimalLatitude'].isnull()) | (mytab['decimalLongitude'].isnull())])
      summary.loc[species_name, 'no_latlong_count'] = count_no_latlong

      mytab = mytab[mytab['decimalLatitude'].notnull()]
      mytab = mytab[mytab['decimalLongitude'].notnull()]

      # We can now count the total number of rows which have been filtered out and add it to the summary table.
      total_removed = count_wrong_taxon_rank + count_no_country_code + count_no_latlong
      summary.loc[species_name, 'total_removed_data_cleaning'] = total_removed

      # The last data cleaning step is to add the total number of rows removed in this 
      # stage to the summary table.
      summary.loc[species_name, "nrecords_clean"] = len(mytab)

      # If there are more than 10000 rows in the clean dataframe take a random
      # sample of 10000 rows
      if len(mytab) > 10000:
        mytab = mytab.sample(10000)

      summary.loc[species_name, "nrecords_after_data_cleaning"] = len(mytab)
      # Finally, we save the filtered table and the summary table.
      mytab.to_csv(file_path + "/filtered_main_tables/" + species_name + ".csv", sep="\t", index=None)
      summary.to_csv(file_path + "/summary_tables/" + species_name + ".csv", sep="\t")

      # add the summary to the big summary table
      all_summary_tables = all_summary_tables.append(summary)

# save the big summary table
all_summary_tables['species_name'] = all_summary_tables.index.values
all_summary_tables.to_csv(file_path + "final_summary_data_cleaning.tsv", sep="\t", index=None)

Warning - the species column for Temnothorax_duloticus has multiple values
